## Project Action Plan
#### Phase One
1.  Develop Hypothesis regarding water quality 
2.  Research parameters, set forth by the EPA determin their relevance and availibility.  Choose aproximately 12 parameters (drop 2 if needed) and number of water plants

3.  Establish time frame and data points for project

#### Phase Two
1.  Download data from state
2.  Review data (shape, discription, headers)
3.  Determin frequency and distribution of missing data
4.  Address missing data

#### Phase Three
1.  Produce  plot consisting of state of California, data collection points, colors indicating water component levels, with two sliders for year and component.
2.  Produce line graph showing five largest cities in CA and change in each component over time frame


In [2]:
#import libraries here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [4]:
#Eliminate ... in panda outputpd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:

#Import Raw Data
CA_Water_Path = 'https://data.cnra.ca.gov/dataset/3f96977e-2597-4baa-8c9b-c433cea0685e/resource/a9e7ef50-54c3-4031-8e44-aa46f3c660fe/download/lab-results.csv'
df = pd.read_csv(CA_Water_Path)

In [5]:
#View Raw Data
df.head()

,STATION_ID,STATION_NAME,FULL_STATION_NAME,STATION_NUMBER,STATION_TYPE,LATITUDE,LONGITUDE,COUNTY_NAME,SAMPLE_CODE,SAMPLE_DATE,SAMPLE_DEPTH,SAMPLE_DEPTH_UNITS,PARAMETER,RESULT,REPORTING_LIMIT,UNITS,METHOD_NAME
0,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.562,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Conductance,3480.0,1.0,uS/cm,EPA 120.1
1,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.562,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Boron,7.7,0.1,mg/L,"Std Method 4500-B, C"
2,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.562,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Calcium,68.0,1.0,mg/L,EPA 215.2
3,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.562,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Chloride,758.0,0.1,mg/L,"Std Method 4500-Cl, B"
4,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.562,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Magnesium,59.0,0.1,mg/L,"Std Method 3500-Mg, E"


In [21]:
max(gw['SAMPLE_DATE'])

Timestamp('2018-10-18 11:15:00')

In [6]:
#Separate out ground water information
gw = df[df['STATION_TYPE'] == 'Surface Water']
#check Data types
gw.dtypes

STATION_ID              int64
STATION_NAME           object
FULL_STATION_NAME      object
STATION_NUMBER         object
STATION_TYPE           object
LATITUDE              float64
LONGITUDE             float64
COUNTY_NAME            object
SAMPLE_CODE            object
SAMPLE_DATE            object
SAMPLE_DEPTH          float64
SAMPLE_DEPTH_UNITS     object
PARAMETER              object
RESULT                float64
REPORTING_LIMIT       float64
UNITS                  object
METHOD_NAME            object
dtype: object

In [22]:
gw.describe()

,STATION_ID,LATITUDE,LONGITUDE,SAMPLE_DEPTH,RESULT,REPORTING_LIMIT
count,2.383690e+06,2.383690e+06,2.383690e+06,1.378686e+06,2.364666e+06,2.375727e+06
mean,8.293433e+03,3.766743e+01,-1.207869e+02,3.276337e+00,1.489256e+02,6.443835e-01
std,1.518463e+04,2.220881e+00,1.895890e+00,1.596003e+01,5.371261e+03,3.227748e+00
min,1.000000e+00,3.254170e+01,-1.244010e+02,-3.500000e-01,0.000000e+00,0.000000e+00
25%,3.970000e+02,3.639060e+01,-1.218200e+02,1.500000e-01,2.000000e-03,5.000000e-02
50%,2.388000e+03,3.797820e+01,-1.215190e+02,1.000000e+00,2.000000e+00,1.000000e-01
75%,5.541000e+03,3.922570e+01,-1.199770e+02,1.000000e+00,3.100000e+01,1.000000e+00
max,4.783800e+04,4.199850e+01,-1.141360e+02,5.490000e+02,5.000000e+06,1.360000e+03


In [7]:
gw['SAMPLE_DATE'] = pd.to_datetime(gw['SAMPLE_DATE'])
gw.dtypes

C:\Users\rick1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


STATION_ID                     int64
STATION_NAME                  object
FULL_STATION_NAME             object
STATION_NUMBER                object
STATION_TYPE                  object
LATITUDE                     float64
LONGITUDE                    float64
COUNTY_NAME                   object
SAMPLE_CODE                   object
SAMPLE_DATE           datetime64[ns]
SAMPLE_DEPTH                 float64
SAMPLE_DEPTH_UNITS            object
PARAMETER                     object
RESULT                       float64
REPORTING_LIMIT              float64
UNITS                         object
METHOD_NAME                   object
dtype: object

In [8]:
gw.head()

,STATION_ID,STATION_NAME,FULL_STATION_NAME,STATION_NUMBER,STATION_TYPE,LATITUDE,LONGITUDE,COUNTY_NAME,SAMPLE_CODE,SAMPLE_DATE,SAMPLE_DEPTH,SAMPLE_DEPTH_UNITS,PARAMETER,RESULT,REPORTING_LIMIT,UNITS,METHOD_NAME
11481,2932,BRUSHY C TRIB A DYER,BRUSHY C TRIB A DYER RD-BRUSHY C,B8933210,Surface Water,37.7672,-121.675,Alameda,WDIS_0897433,1958-10-22 12:25:00,NaN,Feet,Conductance,1030.0,1.0,uS/cm,EPA 120.1
11482,2932,BRUSHY C TRIB A DYER,BRUSHY C TRIB A DYER RD-BRUSHY C,B8933210,Surface Water,37.7672,-121.675,Alameda,WDIS_0897433,1958-10-22 12:25:00,NaN,Feet,Dissolved Boron,1.7,0.1,mg/L,UnkH Boron
11483,2932,BRUSHY C TRIB A DYER,BRUSHY C TRIB A DYER RD-BRUSHY C,B8933210,Surface Water,37.7672,-121.675,Alameda,WDIS_0897433,1958-10-22 12:25:00,NaN,Feet,Dissolved Calcium,51.0,1.0,mg/L,EPA 215.2
11484,2932,BRUSHY C TRIB A DYER,BRUSHY C TRIB A DYER RD-BRUSHY C,B8933210,Surface Water,37.7672,-121.675,Alameda,WDIS_0897433,1958-10-22 12:25:00,NaN,Feet,Dissolved Chloride,100.0,0.1,mg/L,"Std Method 4500-Cl, B"
11485,2932,BRUSHY C TRIB A DYER,BRUSHY C TRIB A DYER RD-BRUSHY C,B8933210,Surface Water,37.7672,-121.675,Alameda,WDIS_0897433,1958-10-22 12:25:00,NaN,Feet,Dissolved Fluoride,1.4,0.1,mg/L,Std Method 10th Ed Fluororide


In [9]:
gw.nunique() 

STATION_ID              4978
STATION_NAME            4826
FULL_STATION_NAME       4922
STATION_NUMBER          4978
STATION_TYPE               1
LATITUDE                4343
LONGITUDE               3007
COUNTY_NAME               58
SAMPLE_CODE           167313
SAMPLE_DATE           143633
SAMPLE_DEPTH             227
SAMPLE_DEPTH_UNITS         2
PARAMETER                386
RESULT                 10478
REPORTING_LIMIT          402
UNITS                     30
METHOD_NAME              277
dtype: int64

In [10]:
pd.value_counts(df['COUNTY_NAME'])

Los Angeles        489528
San Bernardino     321724
Kern               237722
San Joaquin        229141
Riverside          180130
Butte              172406
Ventura            163455
Merced             147545
Sacramento         134371
Contra Costa       128259
Alameda            126004
Fresno             124484
Tehama             107758
Shasta             105869
San Diego          101495
Solano              96360
Lake                90782
Orange              89804
Santa Clara         83863
Kings               83765
Sutter              65800
Yolo                64880
Monterey            61919
Humboldt            61001
Siskiyou            59997
Plumas              56160
Lassen              55681
Stanislaus          53997
San Luis Obispo     49401
Mendocino           48350
Santa Barbara       42720
Imperial            41682
Colusa              41148
Tulare              40047
Modoc               38828
Glenn               38276
Sonoma              32000
Placer              28860
Santa Cruz  

In [11]:
pd.value_counts(df['PARAMETER']).sort_index()

(Aminomethyl)phosphonic acid                          229
*No Lab Analyses (Field Measures Only)              17192
1,1,1,2-Tetrachloroethane                            2828
1,1,1-Trichloroethane                                2946
1,1,2,2-Tetrachloroethane                            2828
1,1,2-Trichloroethane                                2536
1,1,2-Trichlorotrifluoroethane                         25
1,1-Dichloroethane                                   2947
1,1-Dichloroethene                                   2946
1,1-Dichloropropene                                  2828
1,2,3-Trichlorobenzene                               2828
1,2,3-Trichloropropane                               2827
1,2,4-Trichlorobenzene                               2827
1,2,4-Trimethylbenzene                               2835
1,2-Dibromo-3-chloropropane (DBCP)                   2983
1,2-Dibromoethane (EDB)                              2416
1,2-Dichlorobenzene                                  2943
1,2-Dichloroet

In [24]:
gw.groupby(pd.Grouper(key='SAMPLE_DATE', freq='Y')).count()

KeyError: 'The grouper name SAMPLE_DATE is not found'